In [109]:
import torch
import numpy as np
import pandas as pd

In [110]:
val_gt = np.load("../results/ad_simple/val_gt.npy")
val_pred = np.load("../results/ad_simple/val_samples.npy")
test_gt = np.load("../results/ad_simple/test_gt.npy")
test_pred = np.load("../results/ad_simple/test_samples.npy")
val_pred = np.reshape(val_pred,newshape=(val_gt.shape[0],-1,val_gt.shape[1],val_gt.shape[2]))
test_pred = np.reshape(test_pred,newshape=(test_gt.shape[0],-1,test_gt.shape[1],test_gt.shape[2]))

In [111]:
val_gt.shape, val_pred.shape, test_gt.shape, test_pred.shape

((1325, 25, 100), (1325, 10, 25, 100), (879, 25, 100), (879, 10, 25, 100))

In [112]:
low_q = np.quantile(val_pred,0,axis=1)
high_q = np.quantile(val_pred,1,axis=1)
mid_q = np.quantile(val_pred,0.5,axis=1)

In [113]:
low_q.shape, high_q.shape, mid_q.shape

((1325, 25, 100), (1325, 25, 100), (1325, 25, 100))

In [114]:
#convert all variables to tensors
val_gt = torch.tensor(val_gt)
val_pred = torch.tensor(val_pred)
test_gt = torch.tensor(test_gt)
test_pred = torch.tensor(test_pred)
low_q = torch.tensor(low_q)
high_q = torch.tensor(high_q)
mid_q = torch.tensor(mid_q)

In [115]:
criterion = torch.nn.MSELoss(reduction='none')

In [116]:
val_mse = torch.mean(criterion(mid_q,val_gt),dim = -2).view(-1)
val_mse.shape

torch.Size([132500])

In [117]:
torch.quantile(val_mse,0.99)

tensor(7.0067, dtype=torch.float64)

In [118]:
torch.max(criterion(mid_q,val_gt),dim=-2)[0].view(-1)

tensor([7.6245, 7.2617, 7.7339,  ..., 0.6809, 0.6309, 0.6153],
       dtype=torch.float64)

In [119]:
low_q = np.quantile(test_pred,0,axis=1)
high_q = np.quantile(test_pred,1,axis=1)
mid_q = np.quantile(test_pred,0.5,axis=1)
low_q = torch.tensor(low_q)
high_q = torch.tensor(high_q)
mid_q = torch.tensor(mid_q)

In [120]:
test_mse = torch.mean(criterion(mid_q,test_gt),dim = -2).view(-1)
test_mse.shape

torch.Size([87900])

In [121]:
labels = pd.read_csv("../datasets/PSM/test_label.csv",index_col=0)
labels = labels.values.astype(int)

In [122]:
labels = torch.from_numpy(labels)

In [123]:
labels.shape

torch.Size([87841, 1])

In [124]:
combined = torch.cat([val_mse[:132481],test_mse[:labels.shape[0]]])

In [125]:
torch.quantile(combined,0.95)

tensor(3.9162, dtype=torch.float64)

In [126]:
pred = (test_mse>torch.quantile(combined,0.99)).int()

In [127]:
pred

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

In [128]:
pred = pred.reshape(-1,1)

In [129]:
pred = pred[:labels.shape[0]]

In [130]:
pred.shape, labels.shape

(torch.Size([87841, 1]), torch.Size([87841, 1]))

In [131]:
((pred==labels)*labels).sum()

tensor(421)

In [132]:
pred = np.array(pred)
labels = np.array(labels)

In [133]:
pred = pred.reshape(-1)
labels = labels.reshape(-1)

In [134]:
labels[0]

0

In [135]:
anomaly_state = False
for i in range(len(labels)):
    if labels[i] == 1 and pred[i] == 1 and not anomaly_state:
        anomaly_state = True
        for j in range(i, 0, -1):
            if labels[j] == 0:
                break
            else:
                if pred[j] == 0:
                    pred[j] = 1
        for j in range(i, len(labels)):
            if labels[j] == 0:
                break
            else:
                if pred[j] == 0:
                    pred[j] = 1
    elif labels[i] == 0:
        anomaly_state = False
    if anomaly_state:
        pred[i] = 1

In [136]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(labels, pred)
precision, recall, f_score, support = precision_recall_fscore_support(labels, pred,
                                                                        average='binary')
print(
    "Accuracy : {:0.4f}, Precision : {:0.4f}, Recall : {:0.4f}, F-score : {:0.4f} ".format(
        accuracy, precision,
        recall, f_score))

Accuracy : 0.9475, Precision : 0.9994, Recall : 0.8112, F-score : 0.8955 
